<a href="https://colab.research.google.com/github/demyanchuk-nestor/AI_for_Medical_Diagnosis/blob/master/Week-2/UKR_Evaluation_of_Diagnostic_Models_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/demyanchuk-nestor/AI_for_Medical_Diagnosis
%cd 'AI_for_Medical_Diagnosis/Week-2'

# Оцінювання якості алгоритмів штучного інтелекту при діагностиці захворювань


У даній роботі використовуються результати класифікації моделі для медичних зображень, отриманих методами комп’ютерної томографії. Оцінюванні якості алгоритмів штучного інтелекту (ШІ) проведено на наборі тестових даних із 420 комп’ютерних томограм, які характеризують 14 різних патологічних станів. У даній роботі буде проведено оцінювання якості класифікації кожного із досліджуваних захворювань. Для цього будуть використані істинно позитивні, хибно позитивні, істинно негативні та хибно негативні результати, розраховані точність, поширеність, чутливість і специфічність, позитивні прогнозовані значення (Positive predictive value (PPV)) та негативні прогнозовані значення (Negative predictive value (NPV)), побудовані ROC-криві, довірчі інтервали, графік точності запам’ятовування, результати F1 та калібрування.



## Зміст
1. Завантаження вхідних даних та їх візуалізація.
2. Аналіз результатами класифікації та наявністю захворювання.
3. Розрахунок точності алгоритму ШІ.
4. Розрахунок поширеності алгоритму ШІ.
5. Розрахунок чутливості та специфічності алгоритму ШІ.
6. Розрахунок позитивної прогностичної цінності (PPV) та негативної прогностичної цінності (NPV) алгоритму ШІ.
7. Побудова та аналіз ROC-кривої.
8. Розрахунок довірчих інтервалів.
9. Оцінювання кривої точності запам’ятовування.
10. Розрахунок продуктивності алгоритму ШІ із гармонійним середнім значенням точності та запам’ятовування F1.
11. Побудова калібрувальних графіків.


<a name='1'></a>


3.	Перейти за наступними посиланнями, щоб імпортувати всі необхідні пакети, а також спеціальні службові функції за посиланнями:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import util

<a name='2'></a>


4.	У двох файлах CSV під назвами "train_preds.csv" і "valid_preds.csv" збережені дані.
5.	В наступній частині роботи будемо аналізувати прогнозовані значення та справжні мітки класів. Досліджувані дані були у вигляді попередньо класифіковані лікарями у вигляді 14 захворювань
Для завантаження попередньо обчислених результатів моделювання для досліджуваних даних потрібно перейти за посиланням:

In [ ]:
train_results = pd.read_csv("train_preds.csv")
valid_results = pd.read_csv("valid_preds.csv")

# the labels in our dataset
class_labels = ['Cardiomegaly',
 'Emphysema',
 'Effusion',
 'Hernia',
 'Infiltration',
 'Mass',
 'Nodule',
 'Atelectasis',
 'Pneumothorax',
 'Pleural_Thickening',
 'Pneumonia',
 'Fibrosis',
 'Edema',
 'Consolidation']

# the labels for prediction values in our dataset
pred_labels = [l + "_pred" for l in class_labels]

6.	Для генерування міток (y) та результатів прогнозування (pred), необхідно перейти за посиланнями:

In [ ]:
y = valid_results[class_labels].values
pred = valid_results[pred_labels].values

In [ ]:
y.shape, pred.shape

7.	Після переходу за посиланням отримаємо результат у вигляді таблиці.

In [ ]:
# let's take a peek at our dataset
valid_results[np.concatenate([class_labels, pred_labels])].head()

8.	Для отримання інформації у вигляді гістограми про кількість пацієнтів із певним захворюванням необхідно перейти за посиланням:

In [ ]:
plt.xticks(rotation=90)
plt.bar(x = class_labels, height= y.sum(axis=0));

Аналізуючи отримані результати можна зробити висновок, що дані по кількості кожного із захворювань є незбалансованими. Зокрема, у наборі даних є невелика кількість пацієнтів з діагнозом «Грижа» (`Hernia`), яких є лише 3 пацієнти.
Рекомендація: при формуванні досліджуваних баз даних пацієнтів, для ефективного функціонування алгоритмів ШІ, потрібно намагатися, щоб їх кількість була збалансована.



<a name='3'></a>


<a name='3-1'></a>
### 2. Аналіз результатів класифікації та наявністю захворювання.

При оцінюванні якості алгоритмів ШІ при діагностиці захворювань використовують «золотий стандарт».
«Золотий стандарт» (референтний, еталонний) – це результат певного точного, способу визначення наявності або відсутності захворювання, що заслуговує цілковитої довіри дослідників.
«Золотий стандарт» не можна як правило вважати абсолютним еталоном, оскільки із розвитком застосовуваних діагностичних методик та алгоритмів ШІ колишні еталонні методи діагностики можуть бути замінені більш досконалими «золотими стандартами». При використанні діагностичних алгоритмів ШІ, на основі бінарного представлення результатів класифікації, у вигляді істинно позитивних (True positive (TP)), хибно позитивних (False positive (FP)), істинно негативних (True negative (TN)) та хибно негативних (False negative (FN)) результатів зручно використовувати таблицю, яку називають таблицею спряженості, контингентності або невідповідності.


9.	Для підрахунку кількості ТР, FP, FN і TN досліджуваних даних у вигляді 0 та 1, потрібно встановити порогове значення $th$=0,5. Числові значення менші 0,5 будуть заокруглені до нуля, інші (більші 0,5) будуть заокруглені до 1. Для цього необхідно перейти за наступними посиланнями:


In [ ]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def true_positives(y, pred, th=0.5):
    """
    Count true positives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        TP (int): true positives
    """
    TP = 0

    # get thresholded predictions
    thresholded_preds = pred >= th

    # compute TP
    TP = np.sum((y == 1) & (thresholded_preds == 1))

    return TP

def true_negatives(y, pred, th=0.5):
    """
    Count true negatives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        TN (int): true negatives
    """
    TN = 0

    # get thresholded predictions
    thresholded_preds = pred >= th

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # compute TN
    TN = np.sum((y == 0) & (thresholded_preds == 0))

    ### END CODE HERE ###

    return TN

def false_positives(y, pred, th=0.5):
    """
    Count false positives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        FP (int): false positives
    """
    FP = 0

    # get thresholded predictions
    thresholded_preds = pred >= th

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # compute FP
    FP = np.sum((y == 0) & (thresholded_preds == 1))

    ### END CODE HERE ###

    return FP

def false_negatives(y, pred, th=0.5):
    """
    Count false positives.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        FN (int): false negatives
    """
    FN = 0

    # get thresholded predictions
    thresholded_preds = pred >= th

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # compute FN
    FN = np.sum((y == 1) & (thresholded_preds == 0))

    ### END CODE HERE ###

    return FN

У результаті виконання команд отримаємо результати у вигляді таблиці

In [ ]:
# Note: we must explicity import 'display' in order for the autograder to compile the submitted code
# Even though we could use this function without importing it, keep this import in order to allow the grader to work
from IPython.display import display
# Test
df = pd.DataFrame({'y_test': [1,1,0,0,0,0,0,0,0,1,1,1,1,1],
                   'preds_test': [0.8,0.7,0.4,0.3,0.2,0.5,0.6,0.7,0.8,0.1,0.2,0.3,0.4,0],
                   'category': ['TP','TP','TN','TN','TN','FP','FP','FP','FP','FN','FN','FN','FN','FN']
                  })

display(df)
#y_test = np.array([1, 0, 0, 1, 1])
y_test = df['y_test']

#preds_test = np.array([0.8, 0.8, 0.4, 0.6, 0.3])
preds_test = df['preds_test']

threshold = 0.5
print(f"threshold: {threshold}\n")

print(f"""Our functions calcualted:
TP: {true_positives(y_test, preds_test, threshold)}
TN: {true_negatives(y_test, preds_test, threshold)}
FP: {false_positives(y_test, preds_test, threshold)}
FN: {false_negatives(y_test, preds_test, threshold)}
""")

print("Expected results")
print(f"There are {sum(df['category'] == 'TP')} TP")
print(f"There are {sum(df['category'] == 'TN')} TN")
print(f"There are {sum(df['category'] == 'FP')} FP")
print(f"There are {sum(df['category'] == 'FN')} FN")

10. Для отримання та перегляду узагальненої інформації ТР, FP, FN і TN для кожного із досліджуваних захворювань потрібно запустити на виконання наступну команду:

In [ ]:
util.get_performance_metrics(y, pred, class_labels)

В результаті отримаємо таблицю з узагальненою інформацією ТР, FP, FN і TN для кожного із досліджуваних захворювань.


<a name='3-2'></a>
### 3. Розрахунок точності алгоритму ШІ


Використовуючи порогове значення 0,5 для ймовірнісного відсікання прогнозованих значень для всіх класів досліджуваних захворювань обчислимо точність прогнозування моделі:

$$accuracy = \frac{\text{true positives} + \text{true negatives}}{\text{true positives} + \text{true negatives} + \text{false positives} + \text{false negatives}}$$

11.	Для розрахунку точності прогнозування потрібно виконати команди:

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Примітка</b></font>
</summary>
<p>
<ul>
    <li>
Не забудьте встановити значення порогу під час виклику функцій..</li>
</ul>
</p>

In [ ]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def get_accuracy(y, pred, th=0.5):
    """
    Compute accuracy of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        accuracy (float): accuracy of predictions at threshold
    """
    accuracy = 0.0

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # get TP, FP, TN, FN using our previously defined functions
    TP = true_positives(y, pred, th)
    FP = false_positives(y, pred, th)
    TN = true_negatives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # Compute accuracy using TP, FP, TN, FN
    accuracy = (TP + TN)/(TP + TN + FP + FN)

    ### END CODE HERE ###

    return accuracy

In [ ]:
# Test
print("Test case:")

y_test = np.array([1, 0, 0, 1, 1])
print('test labels: {y_test}')

preds_test = np.array([0.8, 0.8, 0.4, 0.6, 0.3])
print(f'test predictions: {preds_test}')

threshold = 0.5
print(f"threshold: {threshold}")

print(f"computed accuracy: {get_accuracy(y_test, preds_test, threshold)}")

#### В результаті отримаємо результат розрахунку точності::

```Python
test labels: {y_test}
test predictions: [0.8 0.8 0.4 0.6 0.3]
threshold: 0.5
computed accuracy: 0.6
```

12.	Для розрахунку точності прогнозування досліджуваної моделі потрібно виконати наступну команду:

In [ ]:
util.get_performance_metrics(y, pred, class_labels, acc=get_accuracy)

При оцінюванні продуктивності запропонованої моделі для прогнозування лише на основі показника точності, то можна зробити висновок, що наша модель не досить точна для виявлення випадків «Infiltration (Проникнення)» (точність 0,657), але досить точна для виявлення «Emphysema (Емфіземи)» (точність 0,889).

13.	Однак, для розрахунку точності моделі, що пацієнт не має «Emphysema (Емфіземи)» потрібно запустити на виконання команду

In [ ]:
get_accuracy(valid_results["Emphysema"].values, np.zeros(len(valid_results)))

Отримуємо точність 97%! Це є кращий результат, ніж прокласифікувала модель на основі глибокого навчання. На основі лише точності ми не можемо зробити висновок про ефективність моделі прогнозування. З цією метою буде розглянуто наступні важливі характеристики чутливість та специфічність, але спочатку розглянемо поширеність.

<a name='3-3'></a>
### 4. Розрахунок поширеності алгоритму ШІ
У медичному контексті поширеність – це частка людей у популяції, які мають певне захворювання.
З точки зору машинного навчання, це частка пацієнтів, які мають певне захворювання. Формула для поширеності має наступний вигляд:


$$prevalence = \frac{1}{N} \sum_{i} y_i$$

where $y_i = 1$ when the example is 'positive' (has the disease).

де уі=1 є прикладом того, що пацієнт має певне захворювання.
14.	Для визначення поширеності потрібно виконати команду


In [ ]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def get_prevalence(y):
    """
    Compute accuracy of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
    Returns:
        prevalence (float): prevalence of positive cases
    """
    prevalence = 0.0

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    prevalence = np.mean(y)

    ### END CODE HERE ###

    return prevalence

Отримаємо результат:

In [ ]:
# Test
print("Test case:\n")

y_test = np.array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1])
print(f'test labels: {y_test}')

print(f"computed prevalence: {get_prevalence(y_test)}")


15.	Для отримання результату в табличній формі для усіх 14 досліджуваних захворювань потрібно запустити команду:

In [ ]:
util.get_performance_metrics(y, pred, class_labels, acc=get_accuracy, prevalence=get_prevalence)

«Hernia (Грижа)» має поширеність 0,002, що є найрідкіснішим серед досліджуваних захворювань у початковому наборі даних.

<a name='3-4'></a>
### 5. Розрахунок чутливості та специфічності алгоритму ШІ
<img src="https://raw.githubusercontent.com/hardik0/AI-for-Medicine-Specialization/master/AI-for-Medical-Diagnosis/Week-2/sens_spec.png" width="30%">

Чутливість і специфічність є двома найбільш відомими показниками, які використовуються для оцінювання якості алгоритмів ШІ, як діагностичних тестів.
Чутливість – це ймовірність того, що наш тест дасть позитивний результат, якщо випадок дійсно позитивний.
Специфічність – це ймовірність того, що результат тесту дасть негативний результат, якщо випадок насправді негативний.
Співвідношення між True positives та False positives можна навести у вигляді істинно позитивного результату, істинно негативного результату, хибно позитивного результату та хибно негативного результату.
Формули для розрахунку чутливості та специфічності мають наступний вигляд:

$$sensitivity = \frac{\text{true positives}}{\text{true positives} + \text{false negatives}}$$

$$specificity = \frac{\text{true negatives}}{\text{true negatives} + \text{false positives}}$$

16.	Розрахуємо чутливість і специфічність для нашої моделі використовуючи перехід за посиланням

In [ ]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def get_sensitivity(y, pred, th=0.5):
    """
    Compute sensitivity of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        sensitivity (float): probability that our test outputs positive given that the case is actually positive
    """
    sensitivity = 0.0

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # get TP and FN using our previously defined functions
    TP = true_positives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # use TP and FN to compute sensitivity
    sensitivity = TP / (TP + FN)

    ### END CODE HERE ###

    return sensitivity

def get_specificity(y, pred, th=0.5):
    """
    Compute specificity of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        specificity (float): probability that the test outputs negative given that the case is actually negative
    """
    specificity = 0.0

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # get TN and FP using our previously defined functions
    TN = true_negatives(y, pred, th)
    FP = false_positives(y , pred, th)

    # use TN and FP to compute specificity
    specificity = TN / (TN + FP)

    ### END CODE HERE ###

    return specificity

In [ ]:
# Test
print("Test case")

y_test = np.array([1, 0, 0, 1, 1])
print(f'test labels: {y_test}\n')

preds_test = np.array([0.8, 0.8, 0.4, 0.6, 0.3])
print(f'test predictions: {preds_test}\n')

threshold = 0.5
print(f"threshold: {threshold}\n")

print(f"computed sensitivity: {get_sensitivity(y_test, preds_test, threshold):.2f}")
print(f"computed specificity: {get_specificity(y_test, preds_test, threshold):.2f}")

#### Отримаємо результат:

```Python
Test case
test labels: [1 0 0 1 1]

test predictions: [0.8 0.8 0.4 0.6 0.3]

threshold: 0.5

computed sensitivity: 0.67
computed specificity: 0.50

```

17.	Для отримання результатів розрахунку чутливості та специфічності для усіх захворюваннях потрібно перейти за посиланням

In [ ]:
util.get_performance_metrics(y, pred, class_labels, acc=get_accuracy, prevalence=get_prevalence,
                        sens=get_sensitivity, spec=get_specificity)

Слід звернути увагу, що специфічність і чутливість не залежать від поширеності позитивного класу в наборі даних. Це пояснюється тим, що статистика обчислюється лише для людей одного класу. Чутливість враховує лише не правильну класифікацію пацієнтів серед тих, що насправді мають певне захворювання. Подібним чином, специфічність враховує лише результат для пацієнтів, що насправді не мають певного захворювання.

<a name='3-5'></a>
### 6. Розрахунок позитивної прогностичної цінності (PPV) та негативної прогностичної цінності (NPV) алгоритму ШІ

Однак з точки зору діагностики чутливість і специфічність не дають змоги комплексно оцінити якість класифікації. Зокрема, чутливість вказує на ймовірність того, що запропонований алгоритм ШІ дасть позитивний результат, враховуючи, що у людини вже є захворювання. Для більш ефективного оцінювання потрібно визначати ймовірність того, що людина хвора, враховуючи, що алгоритм ШІ видає позитивний результат. Це призводить до позитивної прогностичної цінності (PPV) і негативної прогностичної цінності (NPV).
Позитивна прогностична цінність (PPV) — це ймовірність того, що суб’єкти з позитивним скринінговим тестом дійсно мають захворювання.
Негативна прогностична цінність (NPV) — це ймовірність того, що суб’єкти з негативним скринінговим тестом справді не мають захворювання.
У термінах істинно позитивних, істинно негативних, хибно позитивних і хибно негативних результатів:

$$PPV = \frac{\text{true positives}}{\text{true positives} + \text{false positives}}$$

$$NPV = \frac{\text{true negatives}}{\text{true negatives} + \text{false negatives}}$$


PPV=істинно позитивні / істинно позитивні+хибні позитивні результати

NPV= істинно негативні / істинно негативні+хибні негативні значення

18.	Розрахунок PPV та NPV для нашої моделі досліджуваного алгоритму ШІ проведемо за посиланням


In [ ]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def get_ppv(y, pred, th=0.5):
    """
    Compute PPV of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        PPV (float): positive predictive value of predictions at threshold
    """
    PPV = 0.0

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # get TP and FP using our previously defined functions
    TP = true_positives(y, pred, th)
    FP = false_positives(y, pred, th)

    # use TP and FP to compute PPV
    PPV = TP / (TP + FP)

    ### END CODE HERE ###

    return PPV

def get_npv(y, pred, th=0.5):
    """
    Compute NPV of predictions at threshold.

    Args:
        y (np.array): ground truth, size (n_examples)
        pred (np.array): model output, size (n_examples)
        th (float): cutoff value for positive prediction from model
    Returns:
        NPV (float): negative predictive value of predictions at threshold
    """
    NPV = 0.0

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # get TN and FN using our previously defined functions
    TN = true_negatives(y, pred, th)
    FN = false_negatives(y, pred, th)

    # use TN and FN to compute NPV
    NPV = TN / (TN + FN)

    ### END CODE HERE ###

    return NPV

In [ ]:
# Test
print("Test case:\n")

y_test = np.array([1, 0, 0, 1, 1])
print(f'test labels: {y_test}')

preds_test = np.array([0.8, 0.8, 0.4, 0.6, 0.3])
print(f'test predictions: {preds_test}\n')

threshold = 0.5
print(f"threshold: {threshold}\n")

print(f"computed ppv: {get_ppv(y_test, preds_test, threshold):.2f}")
print(f"computed npv: {get_npv(y_test, preds_test, threshold):.2f}")

#### Отримаємо результат у вигляді

```Python
Test case:

test labels: [1 0 0 1 1]
test predictions: [0.8 0.8 0.4 0.6 0.3]

threshold: 0.5

computed ppv: 0.67
computed npv: 0.50
```

19.	Для отримання результатів розрахунку PPV та NPV для усіх досліджуваних захворювань потрібно перейти за посиланням

In [ ]:
util.get_performance_metrics(y, pred, class_labels, acc=get_accuracy, prevalence=get_prevalence,
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv)

Незважаючи на дуже високу чутливість і точність, PPV прогнозів може бути дуже низьким. Це стосується, захворювання Edema, чутливість до якого становить 0,75. Однак, враховуючи, що модель спрогнозувала позитивно, тобто ймовірність того, що у людини є набряк (його PPV), становить лише 0,066.

<a name='3-6'></a>
### 7. Побудова та аналіз ROC-кривої

ROC-крива (англ. receiver operating characteristic, робоча характеристика приймача) – графік, що дає змогу оцінити якість бінарної класифікації та відображає співвідношення між часткою об'єктів від загальної кількості носіїв ознаки, правильно класифікованих до загальної кількості об'єктів, що не несуть ознаки, помилково класифікованих, як такі, що мають ознаку. Також відома як крива похибок. Аналіз класифікацій із застосуванням ROC-кривих називається ROC-аналізом.
Для кількісної інтерпретацію ROC-кривих використовують показник AUC (англ. Area Under ROC Curve, площа під ROC-кривою) – площа, обмежена ROC-кривою і віссю помилкових позитивних класифікацій. Чим вище показник AUC, тим якісніше проведена класифікація, при цьому значення 0,5 демонструє непридатність обраного методу класифікації, що відповідає звичайному вгадуванню.
Крива ROC створюється шляхом побудови графіка істинно позитивних випадків (TPR) проти помилково позитивних випадків (FPR) за різних порогових значень. Ідеальна точка знаходиться у верхньому лівому куті, з коефіцієнтом істинного позитивного результату 1 і коефіцієнтом помилкового позитивного результату 0. Різні точки на кривій генеруються шляхом поступової зміни порогу.

20.	Для побудови ROC-кривих для досліджуваної моделі алгоритму ШІ потрібно перейти за посиланням:


In [ ]:
util.get_curve(y, pred, class_labels)

Отримаємо результати побудови ROC-кривих для усіх досліджуваних захворювань. Площа під кривою ROC також є мірою відповідності якості класифікації. У медичній літературі це число також дає ймовірність того, що випадково вибраний пацієнт, який пережив певний стан, мав вищий показник ризику, ніж пацієнт, який не пережив цієї події. Це підсумовує вихідні дані моделі за всіма пороговими значеннями та забезпечує чітке уявлення про ефективність класифікації даної моделі.

21.	Для розрахунку площі під ROC кривою в таблиці потрібно перейти за посиланням.


In [ ]:
from sklearn.metrics import roc_auc_score
util.get_performance_metrics(y, pred, class_labels, acc=get_accuracy, prevalence=get_prevalence,
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score)

<a name='4'></a>
## 8. Розрахунок довірчих інтервалів.

Довірчий інтервал (англ. confidence interval, CI) — у математичній статистиці є типом інтервальної оцінки, яку обчислюють за даними спостереження, і яка покриває невідомий статистичний параметр із заданою надійністю. Це інтервал, у межах якого з заданою довірчою імовірністю можна очікувати значення оцінюваної (шуканої) випадкової величини.
Для розрахунку довірчих інтервалів потрібно перейти за посиланням:


In [ ]:
def bootstrap_auc(y, pred, classes, bootstraps = 100, fold_size = 1000):
    statistics = np.zeros((len(classes), bootstraps))

    for c in range(len(classes)):
        df = pd.DataFrame(columns=['y', 'pred'])
        df.loc[:, 'y'] = y[:, c]
        df.loc[:, 'pred'] = pred[:, c]
        # get positive examples for stratified sampling
        df_pos = df[df.y == 1]
        df_neg = df[df.y == 0]
        prevalence = len(df_pos) / len(df)
        for i in range(bootstraps):
            # stratified sampling of positive and negative examples
            pos_sample = df_pos.sample(n = int(fold_size * prevalence), replace=True)
            neg_sample = df_neg.sample(n = int(fold_size * (1-prevalence)), replace=True)

            y_sample = np.concatenate([pos_sample.y.values, neg_sample.y.values])
            pred_sample = np.concatenate([pos_sample.pred.values, neg_sample.pred.values])
            score = roc_auc_score(y_sample, pred_sample)
            statistics[c][i] = score
    return statistics

statistics = bootstrap_auc(y, pred, class_labels)

Отримаємо довірчі інтервали на основі обчислення вибіркової статистики.

In [ ]:
util.print_confidence_intervals(class_labels, statistics)

Деякі довірчі інтервали для певних класів набагато ширші, ніж для інших. Грижа, наприклад, має інтервал близько (0,30 - 0,98), що вказує на те, що ми не можемо бути впевнені, що це краще, ніж випадковість (на рівні 0,5).

<a name='5'></a>
## 9. Оцінювання кривої точності запам’ятовування


Оцінювання відгуку (Precision-Recall) є важливим показником якісного прогнозування у випадку дуже незбалансованих класів.
В інформаційному пошуку точність є мірою релевантності результату, що еквівалентно раніше визначеному PPV.
Відгук є мірою того, скільки насправді є релевантних результатів, що еквівалентно визначеній мірі чутливості.
Крива точності запам’ятовування (PRC) показує компроміс між точністю та нагадуванням для різних порогів. Велика площа під кривою PRC означає значний рівень запам’ятовування та високу точність, яка пов’язана з низьким рівнем хибно-позитивних результатів, а висока – з низьким рівнем хибно-негативних результатів.
Високі бали для обох показують, що класифікатор видає точні результати (висока точність), а також забезпечує більшість усіх позитивних результатів (висока запам’ятовуваність).

Для отримання кривих PRC потрібно запустити наступну команду на виконання:


In [ ]:
util.get_curve(y, pred, class_labels, curve='prc')

<a name='6'></a>
## 10. Розрахунок продуктивності алгоритму ШІ із гармонійним середнім значенням точності та запам’ятовування F1.

F1 – це гармонійне середнє значення точності та запам’ятовування. Оцінка F1 досягає найкращого значення при 1 (ідеальна точність і запам’ятовування), а найгірша  при 0.
Для отримання таблиці продуктивності із гармонійним середнім значенням точності та запам’ятовування F1 потрібно виконати наступну команду:


In [ ]:
from sklearn.metrics import f1_score
util.get_performance_metrics(y, pred, class_labels, acc=get_accuracy, prevalence=get_prevalence,
                        sens=get_sensitivity, spec=get_specificity, ppv=get_ppv, npv=get_npv, auc=roc_auc_score,f1=f1_score)

<a name='7'></a>
## 11. Побудова калібрувальних графіків

Виконуючи класифікацію, є необхідність не тільки передбачити мітку класу, але й отримати ймовірність кожної мітки. Ця ймовірність додає певної впевненості в прогнозі. Для спостереження, як згенеровані досліджуваним алгоритмом ШІ ймовірності узгоджуються з реальними ймовірностями, потрібно побудувати калібрувальну криву.
Щоб створити калібрувальний графік, потрібно спочатку розподілити прогнози на фіксовану кількість окремих діапазонів (наприклад 5) між 0 і 1. Потім обчислити точку для кожного діапазону: значення «x» для кожної точки є середнім для ймовірності, яку алгоритм ШІ присвоїв цим точкам, і значення «y» для кожної точки частки справжніх позитивних результатів у цьому наборі. Потім ми наносимо ці точки на лінійний графік. Добре відкалібрована модель має калібрувальну криву, яка майже збігається з лінією y=x.
Бібліотека sklearn має утиліту calibration_curve для створення калібрувального графіка. Для цього потрібно перейти за наступним посиланням


In [ ]:
from sklearn.calibration import calibration_curve
def plot_calibration_curve(y, pred):
    plt.figure(figsize=(20, 20))
    for i in range(len(class_labels)):
        plt.subplot(4, 4, i + 1)
        fraction_of_positives, mean_predicted_value = calibration_curve(y[:,i], pred[:,i], n_bins=20)
        plt.plot([0, 1], [0, 1], linestyle='--')
        plt.plot(mean_predicted_value, fraction_of_positives, marker='.')
        plt.xlabel("Predicted Value")
        plt.ylabel("Fraction of Positives")
        plt.title(class_labels[i])
    plt.tight_layout()
    plt.show()

In [ ]:
plot_calibration_curve(y, pred)

Як показують наведені вище графіки (рис.6), для більшості прогнозів калібрувальний графік нашої моделі не схожий на добре відкалібрований графік. Для виправлення даної негативної ситуації є дуже корисний метод масштабування, який працює шляхом підгонки моделі логістичної регресії до показників досліджуваної моделі. Щоб побудувати таку модель спочатку потрібно завантажити навчальну частину набору даних для створення лінійної моделі. Потім необхідно використати модель для калібрування прогнозів для її тестової частини, перейшовши за наступними посиланнями:

In [ ]:
from sklearn.linear_model import LogisticRegression as LR

y_train = train_results[class_labels].values
pred_train = train_results[pred_labels].values
pred_calibrated = np.zeros_like(pred)

for i in range(len(class_labels)):
    lr = LR(solver='liblinear', max_iter=10000)
    lr.fit(pred_train[:, i].reshape(-1, 1), y_train[:, i])
    pred_calibrated[:, i] = lr.predict_proba(pred[:, i].reshape(-1, 1))[:,1]

In [ ]:
plot_calibration_curve(y[:,], pred_calibrated)

# Щиро вітаємо із завершенням роботи!
Віримо, що Ви відчуєте себе набагато впевненіше у оцінюванні якості алгоритмів ШІ :)